 # Create your own horror movie!!
 ### Create a automatically generated horror movie screenplay,  look for movies from the public domain, and automatically let this script edit a new film.

 ## Pre-production and screenplay  writing

 ### Let's start by web scraping horror movie screenplays

In [ ]:
import requests
import urllib.request
import os
from bs4 import BeautifulSoup

SCREENPLAYS_URLS = ['https://www.simplyscripts.com/genre/horror-scripts.html',
                    'https://www.simplyscripts.com/genre/drama-scripts.html',
                    'https://www.simplyscripts.com/genre/film-noir-scripts.html',
                    'https://www.simplyscripts.com/genre/musical-scripts.html']
FILE_TYPES = ['.pdf', '.txt']

# Folder with screenplays
SCREENPLAYS_FOLDER = 'screenplays'
if not os.path.exists(SCREENPLAYS_FOLDER):
    os.mkdir(SCREENPLAYS_FOLDER)

def scrape_screenplays(urls=SCREENPLAYS_URLS, screenplays_folder=SCREENPLAYS_FOLDER):
    """Saves screenplays into .txt and .pdf files and returns file names

    Keyword Arguments:
        urls {string} -- [Urls to scrap] (default: {SCREENPLAYS_URLS})
        screenplays_folder {string} -- [Path to files] (default: {SCREENPLAYS_FOLDER})

    Returns:
        [list] -- [List of file names]
    """
    file_names = []
    movie_count = 0
    for url in urls:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        for link in soup.find_all('a'):
            file_link = link.get('href')
            try:
                if any(extension in file_link for extension in FILE_TYPES):
                    file = requests.get(file_link)
                    if '<html>' not in file:
                        file_name = file_link.split('/')[-1]
                        file_names.append(file_name)
                        with open(screenplays_folder + '//' + file_name, 'wb') as f:
                            f.write(file.content)
                            movie_count += 1
                            print(str(movie_count))
            except Exception as identifier:
                print(identifier)
    return file_names

file_names = scrape_screenplays()


 ### We continue to parse the names and get more info about the movies using IMDB's API
 ### Preliminary inspection shows a pattern '%20' that should be replaced with a blank space

In [ ]:
import pandas as pd
from imdb import IMDb # documentation on https://imdbpy.github.io/

imdb_object = IMDb()


files_data = []
for file_name in file_names:
    movie_name = file_name[:-4].replace('%20', '') # Erase extension and %20 characters
    print(movie_name)
    movie_list = imdb_object.search_movie(movie_name)
    title = movie_name
    rating = None
    year = None #TODO: TURN INTO INT

    if movie_list:
        movie_id = movie_list[0].getID() # Assumes first movie on search is the correct one
        movie = imdb_object.get_movie(movie_id)
        # TODO: REFACTOR
        title = movie['title'] if movie.has_key('title') else movie_name
        rating = movie['rating'] if movie.has_key('rating') else None
        year = movie['year'] if movie.has_key('year') else None

    files_data.append({'file_name': file_name,
                        'title': title,
                        'rating': rating,
                        'year': year
                        })

# # # Save dataframe to csv
# # movies_df = pd.DataFrame(files_data)
# # movies_df.to_csv('screenplays_extra_data.csv') #TODO: Refactor screenplays


 ### We filter further our new dataset (and do visualizations)
 ### We found a significant negative relation between year of publication and rating
 ### Reasons for it are not clear given the small dataset, it could be that only "bad"
 ### recent horror movies have their scripts online, unlike older movies.

In [ ]:
# Read screenplay files from local folder
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
movies_df = pd.read_csv('screenplays_extra_data.csv')
movies_df = movies_df.dropna() # Drop if any row value is NaN
# #%%
# # Ratings
plt.xlim(1, 10)
plt.xlabel('Rating')
plt.ylabel('Freq')
plt.title('Ratings histogram')
movies_df['rating'].hist()


In [ ]:
# Year
plt.xlabel('Year')
plt.ylabel('Freq')
plt.title('Year histogram')
movies_df['year'].hist()

In [ ]:
# Relationship between year and rating (in my small dataset of films with script)
import scipy.stats
ax = sns.regplot(x='year', y='rating', data=movies_df)
scipy.stats.linregress(movies_df['year'], movies_df['rating'])


 ### A recurrent neural network is trained with the screenplays, to generate a new one

In [ ]:
import tensorflow as ts
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

SCREENPLAYS_FOLDER = 'screenplays'

file_names = movies_df['file_name']

In [ ]:
html_count = 0
no_html_count = 0
has_html_column = []
is_txt = []
for file_name in file_names:

    text_has_html = False
    text = open(SCREENPLAYS_FOLDER + '//' + file_name, 'rb').readlines()

    for line in text:
        if '<html>' in str(line):
            text_has_html = True
            break
    is_txt.append(True if 'txt' in file_name else False)
    has_html_column.append(text_has_html)

In [ ]:
movies_df['has_html'] = has_html_column
movies_df['is_txt'] = is_txt

In [ ]:
movies_df.to_csv('screenplays_extra_data2.csv')

In [ ]:

file_names = movies_df[(~movies_df['has_html']) & (movies_df['is_txt'])]['file_name']


In [ ]:
test_file = file_names.iloc[5]

In [ ]:

# Read, then decode for py2 compat.
# TODO: TURN IT INTO STRING HERE
text = open(SCREENPLAYS_FOLDER + '//' + test_file, 'rb').read().decode('utf-8','ignore').encode("utf-8")
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))



In [ ]:
# The unique characters in the file
vocab = sorted(set(str(text)))
print ('{} unique characters'.format(len(vocab)))

In [ ]:
# RNN
# Vectorize text
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

In [ ]:
text_as_int = np.array([char2idx[c] for c in str(text)])


In [ ]:
import tensorflow as tf
text = str(text).replace('b', '')
# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [ ]:
# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))



In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)


In [ ]:
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))


In [ ]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))



In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset


In [ ]:
# Build the model
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024


In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)



In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")



In [ ]:
model.summary()



In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()



In [ ]:
sampled_indices


In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))

print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))



In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())



In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
import os
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)


In [ ]:
EPOCHS=10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])



In [ ]:
# Generate text
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))



In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 2.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))



In [ ]:
print(generate_text(model, start_string=u"ANDY"))

